### Dataset

In [1]:
from src.datasets_and_annotations.segmentsai_handler import SegmentsAIHandler
from src.pipelines import model_assist_label_pipeline as pipe 
import config

In [2]:
segmentsai_handler = SegmentsAIHandler()
DATASET_NAME = 'etaylor/stigmas_color_dataset'


In [4]:
dataset = segmentsai_handler.client.get_dataset(DATASET_NAME)

In [16]:

# Replace 'user/dataset' and 'release_name' with your dataset's details
release = segmentsai_handler.client.get_release(DATASET_NAME, 'v0.2')


In [28]:
from segments import SegmentsDataset
from segments.utils import export_dataset

annotation_format = "coco-instance"
local_dataset_dir = "/home/etaylor/code_projects/thesis/segments/stigmas_colors_dataset_semantic"

dataset = SegmentsDataset(release, filter_by=['labeled'])

# Export to COCO
export_dataset(dataset, export_format=annotation_format, export_folder=local_dataset_dir)

Initializing dataset...
Preloading all samples. This may take a while...


100%|██████████| 84/84 [00:10<00:00,  7.68it/s]


Initialized dataset with 84 images.
Exporting dataset. This may take a while...


100%|██████████| 84/84 [00:02<00:00, 41.44it/s]

Exported to /home/etaylor/code_projects/thesis/segments/stigmas_colors_dataset_semantic/export_coco-instance_etaylor_stigmas_color_dataset_v0.2.json. Images in segments/etaylor_stigmas_color_dataset/v0.2


('/home/etaylor/code_projects/thesis/segments/stigmas_colors_dataset_semantic/export_coco-instance_etaylor_stigmas_color_dataset_v0.2.json',
 'segments/etaylor_stigmas_color_dataset/v0.2')

### Fine Tune Small YOLO Models

In [3]:
from src.segmentation.framework_handlers import ultralytics_handler
ultralytics_handler.validate_version_and_gpu()

Ultralytics 8.3.56 🚀 Python-3.9.19 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
Setup complete ✅ (12 CPUs, 125.2 GB RAM, 15.6/336.4 GB disk)


In [4]:
from ultralytics import settings
import config

# Update a setting
yolo_runs_dir = "/home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/color_segmentation_runs"
settings.update(
    {
        'runs_dir': yolo_runs_dir,
        'weights_dir': f"{yolo_runs_dir}/weights",
        'datasets_dir': f"{yolo_runs_dir}/datasets",
    }
)

# View all settings
print(settings)

JSONDict("/home/etaylor/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/color_segmentation_runs/datasets",
  "weights_dir": "/home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/color_segmentation_runs/weights",
  "runs_dir": "/home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/color_segmentation_runs",
  "uuid": "01de958df926b4e70f7bb00e8bdc993b48b07bc8d023fc13b26874ee84610e6f",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": true,
  "wandb": false,
  "vscode_msg": true
}


To clean the GPU memory run this

In [5]:
import torch
import gc

gc.collect()
torch.cuda.empty_cache()

In [6]:
import os
import json
from ultralytics import YOLO
import torch
import gc

# Path to the dataset YAML configuration
yaml_file_path = "/home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/yaml/stigmas_color_dataset_yolo.yaml"

# Paths to model checkpoints and fine-tuned checkpoint storage
pre_trained_checkpoint_dir = "/home/etaylor/code_projects/thesis/checkpoints/stigmas_segmentation/yolo/pre_trained"
fine_tuned_checkpoint_dir = "/home/etaylor/code_projects/thesis/checkpoints/stigmas_color_segmentation/yolo/fine_tuned"

# YOLO segmentation model checkpoints
available_models_checkpoints = {
    "yolov8n_seg": os.path.join(pre_trained_checkpoint_dir, "yolov8n-seg.pt"),
    "yolov8s_seg": os.path.join(pre_trained_checkpoint_dir, "yolov8s-seg.pt"),
    "yolov8m_seg": os.path.join(pre_trained_checkpoint_dir, "yolov8m-seg.pt"),
    # "yolov8l_seg": os.path.join(pre_trained_checkpoint_dir, "yolov8l-seg.pt"),
    # "yolov8x_seg": os.path.join(pre_trained_checkpoint_dir, "yolov8x-seg.pt"),
    "yolov11n_seg": os.path.join(pre_trained_checkpoint_dir, "yolo11n-seg.pt"),
    "yolov11s_seg": os.path.join(pre_trained_checkpoint_dir, "yolo11s-seg.pt"),
    "yolov11m_seg": os.path.join(pre_trained_checkpoint_dir, "yolo11m-seg.pt"),
    # "yolov11l_seg": os.path.join(pre_trained_checkpoint_dir, "yolo11l-seg.pt"),
    # "yolov11x_seg": os.path.join(pre_trained_checkpoint_dir, "yolo11x-seg.pt")
}

# Ensure fine-tuned checkpoint directory exists
os.makedirs(fine_tuned_checkpoint_dir, exist_ok=True)

for model_name, checkpoint_name in available_models_checkpoints.items():
    print(f"Training model: {model_name}")

    # Load the pre-trained model checkpoint from the original path
    model = YOLO(checkpoint_name)

    # Train the model
    results = model.train(data=yaml_file_path,
                        epochs=50,
                        batch=8,
                        augment=True,  # Enables built-in augmentations
                        degrees=10,  # Rotation (±10 degrees)
                        translate=0.1,  # Random shift (10% of image size)
                        scale=0.5,  # Random rescaling factor (0.5x to 1.5x)
                        flipud=0.5,  # Vertical flip probability
                        fliplr=0.5,  # Horizontal flip probability
                        hsv_h=0.015,  # Random hue change
                        hsv_s=0.7,  # Random saturation change
                        hsv_v=0.4,  # Random brightness change
                        project='/home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/color_segmentation_runs',  # Custom root directory
                        name=model_name  # Specific subdirectory for this run
                        )


    print("Fine-tuning complete. Results saved.")
    
    # Save the fine-tuned checkpoint
    fine_tuned_checkpoint_path = os.path.join(fine_tuned_checkpoint_dir, f"{model_name}_fine_tuned.pt")
    model.save(fine_tuned_checkpoint_path)
    print(f"Fine-tuned model saved to: {fine_tuned_checkpoint_path}")

    gc.collect()
    torch.cuda.empty_cache()

Training model: yolov8n_seg
Ultralytics 8.3.56 🚀 Python-3.9.19 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
engine/trainer: task=segment, mode=train, model=/home/etaylor/code_projects/thesis/checkpoints/stigmas_segmentation/yolo/pre_trained/yolov8n-seg.pt, data=/home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/yaml/stigmas_color_dataset_yolo.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/color_segmentation_runs, name=yolov8n_seg, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, 

train: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_stigmas_color_dataset/yolo_formatted/labels/train.cache... 68 images, 0 backgrounds, 0 corrupt: 100%|██████████| 68/68 [00:00<?, ?it/s]
/home/etaylor/.conda/envs/detectron_fastai/lib/python3.9/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
val: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_stigmas_color_dataset/yolo_formatted/labels/val.cache... 16 images, 0 backgrounds, 0 corrupt: 100%|██████████| 16/16 [00:00<?, ?it/s]
/home/etaylor/.conda/envs/detectron_fastai/lib/python3.9/site-packages/torch/uti

Plotting labels to /home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/color_segmentation_runs/yolov8n_seg/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/color_segmentation_runs/yolov8n_seg
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       1/50      1.63G      1.737      5.529      3.111      1.895         13        640: 100%|██████████| 9/9 [00:05<00:00,  1.52it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.28it/s]

                   all         16         18    0.00379          1     0.0909     0.0471    0.00271      0.738     0.0376     0.0109


...


50 epochs completed in 0.092 hours.
Optimizer stripped from /home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/color_segmentation_runs/yolov11m_seg/weights/last.pt, 45.2MB
Optimizer stripped from /home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/color_segmentation_runs/yolov11m_seg/weights/best.pt, 45.2MB

Validating /home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/color_segmentation_runs/yolov11m_seg/weights/best.pt...
Ultralytics 8.3.56 🚀 Python-3.9.19 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
YOLO11m-seg summary (fused): 330 layers, 22,336,854 parameters, 0 gradients, 123.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?, ?it/s]

WARNING ⚠️ Model does not support 'augment=True', reverting to single-scale prediction.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.32it/s]


                   all         16         18      0.897       0.81      0.878      0.643      0.897       0.81      0.878      0.431
                 green          8          8      0.794       0.75      0.761      0.502      0.794       0.75      0.761      0.379
                orange         10         10          1       0.87      0.995      0.784          1       0.87      0.995      0.483
Speed: 0.2ms preprocess, 22.1ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to /home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/color_segmentation_runs/yolov11m_seg
Fine-tuning complete. Results saved.
Fine-tuned model saved to: /home/etaylor/code_projects/thesis/checkpoints/stigmas_color_segmentation/yolo/fine_tuned/yolov11m_seg_fine_tuned.pt


In [8]:
# Perform evaluation on the test set
results = model.val(data='/home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/yaml/stigmas_color_dataset_yolo.yaml', split='val')

Ultralytics 8.3.56 🚀 Python-3.9.19 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)


val: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_stigmas_color_dataset/yolo_formatted/labels/val.cache... 16 images, 0 backgrounds, 0 corrupt: 100%|██████████| 16/16 [00:00<?, ?it/s]
/home/etaylor/.conda/envs/detectron_fastai/lib/python3.9/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/2 [00:00<?, ?it/s]

WARNING ⚠️ Model does not support 'augment=True', reverting to single-scale prediction.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  50%|█████     | 1/2 [00:00<00:00,  1.88it/s]

WARNING ⚠️ Model does not support 'augment=True', reverting to single-scale prediction.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.02it/s]


                   all         16         18      0.886      0.814      0.877       0.64      0.886      0.814      0.877      0.429
                 green          8          8      0.773       0.75      0.759      0.504      0.773       0.75      0.759       0.38
                orange         10         10          1      0.878      0.995      0.776          1      0.878      0.995      0.479
Speed: 0.4ms preprocess, 98.7ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /home/etaylor/code_projects/thesis/src/stigmas_detection/segmentation/yolo/color_segmentation_runs/yolov11m_seg3


In [9]:
results

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f68b9b57280>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.04104